In [1]:
# Function to check if a state is valid
def is_valid(state):
    m_left, c_left, b_pos, m_right, c_right = state
    if m_left < 0 or c_left < 0 or m_right < 0 or c_right < 0:
        return False
    if m_left > 3 or c_left > 3 or m_right > 3 or c_right > 3:
        return False
    if (c_left > m_left > 0) or (c_right > m_right > 0):
        return False
    return True
# Function to generate the next valid states from the current state
def next_states(state):
    m_left, c_left, b_pos, m_right, c_right = state
    if b_pos == 'left':
        moves = [(2, 0), (0, 2), (1, 1), (1, 0), (0, 1)]
        next_states = [(m_left-m, c_left-c, 'right', m_right+m, c_right+c) for m, c in moves]
    else:
        moves = [(-2, 0), (0, -2), (-1, -1), (-1, 0), (0, -1)]
        next_states = [(m_left+m, c_left+c, 'left', m_right-m, c_right-c) for m, c in moves]
    return [state for state in next_states if is_valid(state)]
# Depth-First Search algorithm with depth limit
def dfs_limit(start_state, depth_limit):
    frontier = []  # Use a stack for DFS
    frontier.append([start_state])
    explored = set()
    while frontier:
        path = frontier.pop()  # Pop the last element from the stack
        current_state = path[-1]
        if current_state == (0, 0, 'right', 3, 3):
            return path
        if len(path) <= depth_limit:  # Check depth limit
            for next_state in next_states(current_state):
                if next_state not in explored:
                    new_path = path + [next_state]
                    frontier.append(new_path)
                    explored.add(next_state)
    return None
# Iterative Deepening Depth-First Search algorithm
def iddfs(start_state):
    depth_limit = 0
    while True:
        result = dfs_limit(start_state, depth_limit)
        if result:
            return result
        depth_limit += 1
# Testing the algorithm with the initial state (3, 3, 'left', 0, 0)
start_state = (3, 3, 'left', 0, 0)
path = iddfs(start_state)
# Printing the path from the initial state to the goal state
if path:
    for state in path:
        print(state)
else:
    print("No solution found.") 

(3, 3, 'left', 0, 0)
(2, 2, 'right', 1, 1)
(1, 1, 'left', 2, 2)
(0, 0, 'right', 3, 3)


In [5]:
import copy

# Generate valid moves for a given state
def get_possible_moves(state):
    left_missionaries, left_cannibals, right_missionaries, right_cannibals, boat_position = state
    possible_moves = []

    # Define possible transitions for moves
    transitions = [
        (1, 0),  # One missionary
        (2, 0),  # Two missionaries
        (0, 1),  # One cannibal
        (0, 2),  # Two cannibals
        (1, 1),  # One missionary and one cannibal
    ]

    # Generate new states from valid moves
    for transition in transitions:
        m, c = transition  # Missionaries and cannibals to move
        if boat_position == 'left':
            new_left_m = left_missionaries - m
            new_left_c = left_cannibals - c
            new_right_m = right_missionaries + m
            new_right_c = right_cannibals + c
            new_boat_position = 'right'
        else:
            new_left_m = left_missionaries + m
            new_left_c = left_cannibals + c
            new_right_m = right_missionaries - m
            new_right_c = right_cannibals - c
            new_boat_position = 'left'

        # Validate new state based on the problem's constraints
        if (
            new_left_m >= 0
            and new_left_c >= 0
            and new_right_m >= 0
            and new_right_c >= 0
            and (new_left_m == 0 or new_left_m >= new_left_c)
            and (new_right_m == 0 or new_right_m >= new_right_c)
        ):
            possible_moves.append(
                (
                    new_left_m,
                    new_left_c,
                    new_right_m,
                    new_right_c,
                    new_boat_position,
                )
            )

    return possible_moves

# Depth-Limited Search (DLS) with backtracking
def depth_limited_search(state, goal, limit, path):
    if state == goal:
        return path  # Goal reached

    if len(path) >= limit:
        return None  # Exceeded depth limit

    # Explore possible moves within the depth limit
    for new_state in get_possible_moves(state):
        if new_state not in path:
            new_path = path + [new_state]
            result = depth_limited_search(new_state, goal, limit, new_path)
            if result:
                return result

    return None  # No solution found within the depth limit

# Iterative Deepening Depth-First Search (IDDFS)
def iddfs(start, goal, max_depth):
    # Try increasing depth limits from 1 to max_depth
    for depth in range(1, max_depth + 1):
        result = depth_limited_search(start, goal, depth, [start])
        if result:
            return result  # Solution found
    return None  # No solution found within the depth limit

# Example usage
if __name__ == "__main__":
    # Starting state: All missionaries and cannibals on the left
    start_state = (3, 3, 0, 0, 'left')
    # Goal state: All missionaries and cannibals on the right
    goal_state = (0, 0, 3, 3, 'right')

    # Use IDDFS to solve the problem with a maximum depth of 10
    solution = iddfs(start_state, goal_state, 20)

    if solution:
        print("Solution found:")
        for step, state in enumerate(solution):
            print(f"Step {step}: {state}")
    else:
        print("No solution found within the depth limit.")


Solution found:
Step 0: (3, 3, 0, 0, 'left')
Step 1: (3, 1, 0, 2, 'right')
Step 2: (3, 2, 0, 1, 'left')
Step 3: (3, 0, 0, 3, 'right')
Step 4: (3, 1, 0, 2, 'left')
Step 5: (1, 1, 2, 2, 'right')
Step 6: (2, 2, 1, 1, 'left')
Step 7: (0, 2, 3, 1, 'right')
Step 8: (0, 3, 3, 0, 'left')
Step 9: (0, 1, 3, 2, 'right')
Step 10: (1, 1, 2, 2, 'left')
Step 11: (0, 0, 3, 3, 'right')
